In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [ ]:

spark = SparkSession.builder.appName("SC_factPurchaseRequisition").getOrCreate()

staging_ekpo = "abfss://staging@datalake.dfs.core.windows.net/sap/ekpo/"
staging_ekko = "abfss://staging@datalake.dfs.core.windows.net/sap/ekko/"


In [ ]:

ekpo_df = spark.read.parquet(staging_ekpo)
ekko_df = spark.read.parquet(staging_ekko)

df = ekpo_df.join(ekko_df, "EBELN", "left")


In [ ]:

spark.sql("""
CREATE TABLE IF NOT EXISTS sc.factPurchaseRequisition (
    EBELN STRING,
    EBELP STRING,
    MATNR STRING,
    WERKS STRING,
    MENGE DECIMAL(18,2)
)
USING DELTA
""")


In [ ]:

df.select(
    "EBELN",
    "EBELP",
    "MATNR",
    "WERKS",
    "MENGE"
).write.mode("overwrite").saveAsTable("sc.factPurchaseRequisition")


In [ ]:

spark.sql("VACUUM sc.factPurchaseRequisition RETAIN 168 HOURS")
